# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [4]:
import pandas as pd
import numpy as np

recipes = pd.read_csv("recipes.tsv", sep="\t")

# Sweet tooth persona
def sweet_tooth(recipe_name):
    name = recipe_name.lower()
    if any(word in name for word in ["cake", "cookie", "brownie", "chocolate", "pie", "pudding", "dessert", "candy"]):
        return 5
    elif any(word in name for word in ["fruit", "apple", "banana", "berry", "honey", "jam"]):
        return 4
    elif any(word in name for word in ["bread", "pasta", "rice"]):
        return 3
    elif any(word in name for word in ["chicken", "beef", "pork", "fish", "soup", "stew"]):
        return 2
    elif any(word in name for word in ["salad", "broccoli", "spinach", "cabbage", "pepper", "onion"]):
        return 1
    else:
        return 3

recipes["rating_raw"] = recipes["recipe_title"].apply(sweet_tooth)

# Map raw ratings (1-5) to 0-1 range
recipes["rating"] = (recipes["rating_raw"] - 1) * 0.25

ratings = recipes[["recipe_slug", "rating"]]

# Ensure at least 10 ratings of 0 and 10 of 1
num_zeros = (ratings["rating"] == 0).sum()
num_ones = (ratings["rating"] == 1).sum()

# If not enough zeros or ones
if num_zeros < 10:
    zero_candidates = ratings[ratings["rating"] > 0].sample(10 - num_zeros, random_state=0).index
    ratings.loc[zero_candidates, "rating"] = 0

if num_ones < 10:
    one_candidates = ratings[ratings["rating"] < 1].sample(10 - num_ones, random_state=1).index
    ratings.loc[one_candidates, "rating"] = 1

output_path = "ratings.tsv"
ratings.to_csv(output_path, sep="\t", index=False)

ratings.head()

,recipe_slug,rating
0,falafel,0.50
1,spamburger,0.50
2,bacon-fried-rice,0.00
3,chicken-fingers,0.25
4,apple-crisp,0.75


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [5]:
# YOUR CHANGES HERE

tags = pd.read_csv("recipe-tags.tsv", sep="\t")

features = tags.assign(value=1).pivot_table(
    index="recipe_slug", columns="recipe_tag", values="value", fill_value=0
).reset_index()

features["bias"] = 1
cols = ["recipe_slug", "bias"] + [c for c in features.columns if c not in ["recipe_slug", "bias"]]
features = features[cols]

# Align order with ratings
features = features.set_index("recipe_slug").reindex(ratings["recipe_slug"]).reset_index()

features_path = "features.tsv"
features.to_csv(features_path, sep="\t", index=False)

features.head()

recipe_tag,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,spamburger,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,bacon-fried-rice,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,chicken-fingers,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,apple-crisp,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [12]:
# YOUR CHANGES HERE

from sklearn.linear_model import Ridge

ratings = pd.read_csv("ratings.tsv", sep="\t")
features = pd.read_csv("features.tsv", sep="\t")

X = features.drop(columns=["recipe_slug"]).values
y = ratings["rating"].values

ridge = Ridge(alpha=1.0, fit_intercept=False)
ridge.fit(X, y)

y_pred = ridge.predict(X)

coef = ridge.coef_

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [13]:
# YOUR CHANGES HERE

feature_names = features.drop(columns=["recipe_slug"]).columns

coef_df = pd.DataFrame({
    "recipe_tag": feature_names,
    "coefficient": ridge.coef_
})

model_path = "model.tsv"
coef_df.to_csv(model_path, sep="\t", index=False)

coef_df.head()

,recipe_tag,coefficient
0,bias,0.381034
1,alfredo,-0.021786
2,almond,0.063498
3,american,0.090574
4,appetizer,0.003991


Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [9]:
# YOUR CHANGES HERE

score_estimates = ridge.predict(X)

estimates_df = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_estimate": score_estimates
})

estimates_path = "estimates.tsv"
estimates_df.to_csv(estimates_path, sep="\t", index=False)

estimates_df.head()

,recipe_slug,score_estimate
0,falafel,0.482525
1,spamburger,0.494947
2,bacon-fried-rice,0.058390
3,chicken-fingers,0.261292
4,apple-crisp,0.741771


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [10]:
# YOUR CHANGES HERE

X = features.drop(columns=["recipe_slug"]).values
y = ratings["rating"].values

lambda_val = 1.0
A = X.T @ X + lambda_val * np.identity(X.shape[1])
A_inv = np.linalg.inv(A)

theta_hat = ridge.coef_

alpha = 2.0
estimates = X @ theta_hat
conf_terms = np.array([np.sqrt(x @ A_inv @ x) for x in X])
upper_bounds = estimates + alpha * conf_terms

bounds_df = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_bound": upper_bounds
})

bounds_path = "bounds.tsv"
bounds_df.to_csv(bounds_path, sep="\t", index=False)

bounds_df.head()

,recipe_slug,score_bound
0,falafel,2.248780
1,spamburger,2.389283
2,bacon-fried-rice,1.924547
3,chicken-fingers,2.055408
4,apple-crisp,2.518113


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [11]:
# YOUR CHANGES HERE

X = features.drop(columns=["recipe_slug"]).values
slugs = features["recipe_slug"].tolist()
y = ratings["rating"].values

lambda_val = 1.0
alpha = 2.0
d = X.shape[1]

A = lambda_val * np.identity(d)
A_inv = np.linalg.inv(A)
b = np.zeros(d)

recommendations = []

for t in range(100):
    theta_hat = A_inv @ b
    
    estimates = X @ theta_hat
    conf_terms = np.sqrt(np.sum(X @ A_inv * X, axis=1))
    ucbs = estimates + alpha * conf_terms
    
    choice_idx = np.argmax(ucbs)
    choice_slug = slugs[choice_idx]
    score_bound = ucbs[choice_idx]
    reward = y[choice_idx]
    
    recommendations.append((choice_slug, score_bound, reward))
    
    x = X[choice_idx].reshape(-1, 1)
    Ax = A_inv @ x
    denom = 1.0 + (x.T @ Ax).item()
    A_inv = A_inv - (Ax @ Ax.T) / denom
    
    b += reward * X[choice_idx]

recs_df = pd.DataFrame(recommendations, columns=["recipe_slug", "score_bound", "reward"])
recs_path = "recommendations.tsv"
recs_df.to_csv(recs_path, sep="\t", index=False)

recs_df.head()

,recipe_slug,score_bound,reward
0,apple-crumble,7.483315,0.75
1,ma-la-chicken,7.242589,0.25
2,quesadillas,7.195355,0.50
3,ramen,7.204248,0.50
4,pain-au-chocolat,6.966548,0.50


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.